In [2]:
import mysql.connector
from getpass import getpass

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=getpass("Enter your password: "),
  database="leetcode"
)



Enter your password:  ········


1. Hierarchical Material Efficiency
Identify products where raw material usage exceeds the average of their product type, and rank them within their industry using a moving average of CO2 emissions.

In [11]:
mycursor = mydb.cursor()

sql = """WITH ProductAverages AS (
    SELECT 
        Product_Type,
        AVG(Raw_Material_Usage_kg) AS Avg_Material_Usage
    FROM green_supply_chain
    GROUP BY Product_Type
)
SELECT 
    g.ID,
    g.Product_Type,
    g.Raw_Material_Usage_kg,
    AVG(g.CO2_Emissions_kg) OVER (
        PARTITION BY g.Product_Type 
        ORDER BY g.ID 
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Moving_Avg_CO2
FROM green_supply_chain g
JOIN ProductAverages pa ON g.Product_Type = pa.Product_Type
WHERE g.Raw_Material_Usage_kg > pa.Avg_Material_Usage;"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)

# from prettytable import PrettyTable
# table = PrettyTable()
# table.field_names = [desc[0] for desc in mycursor.description]
# for row in myresult:
#     table.add_row(row)
# print(table)

      ID    Product_Type  Raw_Material_Usage_kg  Moving_Avg_CO2
0     16         Apparel             169.390636      861.245738
1     21         Apparel             122.342333      887.493884
2     44         Apparel             130.130215      695.092154
3     58         Apparel             180.630315      533.164084
4     65         Apparel             157.350247      443.304000
..   ...             ...                    ...             ...
500  953  Pharmaceutical             154.455066      741.284390
501  961  Pharmaceutical             192.319562      657.310585
502  971  Pharmaceutical             150.097845      641.873253
503  976  Pharmaceutical             159.153143      558.100330
504  988  Pharmaceutical             131.068281      392.425161

[505 rows x 4 columns]


2. Nested Sustainability Tiers
Classify products into tiers (Gold/Silver/Bronze) based on renewable energy usage percentiles, then calculate the median cost for each tier.

In [16]:
mycursor = mydb.cursor()

sql = """WITH EnergyTiers AS (
    SELECT 
        *,
        NTILE(3) OVER (ORDER BY Renewable_Energy_pct DESC) AS Tier
    FROM green_supply_chain
)
SELECT 
    CASE Tier 
        WHEN 1 THEN 'Gold' 
        WHEN 2 THEN 'Silver' 
        ELSE 'Bronze' 
    END AS Tier_Group,
    AVG(cost_dollar) AS Median_Cost
FROM EnergyTiers
GROUP BY Tier;
"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


  Tier_Group  Median_Cost
0       Gold  2545.893373
1     Silver  2475.407028
2     Bronze  2486.851680


3. Recursive Supply Chain Impact
Calculate cumulative CO2 emissions for products with transport distances forming a chain (each product’s transport distance ≥ previous product’s distance).

In [ ]:
mycursor = mydb.cursor()

sql = """WITH RECURSIVE EmissionChain AS (
    SELECT 
        ID,
        Transport_Distance_km,
        CO2_Emissions_kg AS Cumulative_CO2
    FROM green_supply_chain
    UNION ALL
    SELECT 
        g.ID,
        g.Transport_Distance_km,
        ec.Cumulative_CO2 + g.CO2_Emissions_kg
    FROM green_supply_chain g
    JOIN EmissionChain ec ON g.Transport_Distance_km >= ec.Transport_Distance_km
)
SELECT MAX(Cumulative_CO2) AS Max_Chain_Impact FROM EmissionChain;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


4. Pivoted Renewable Energy Analysis
Display renewable energy percentages across industries in a cross-tab format (columns: 0-30%, 30-60%, 60-100%).

In [ ]:
mycursor = mydb.cursor()

sql = """SELECT 
    Product_Type,
    COUNT(CASE WHEN Renewable_Energy_Percentage <= 30 THEN 1 END) AS Low_Renewables,
    COUNT(CASE WHEN Renewable_Energy_Percentage BETWEEN 30 AND 60 THEN 1 END) AS Medium_Renewables,
    COUNT(CASE WHEN Renewable_Energy_Percentage > 60 THEN 1 END) AS High_Renewables
FROM green_supply_chain
GROUP BY Product_Type;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


5. Optimized Pagination for Large Datasets
Retrieve rows 100-110 efficiently without OFFSET for a dashboard displaying high-waste products.

In [ ]:
mycursor = mydb.cursor()

sql = """SELECT *
FROM green_supply_chain
WHERE Waste_Generated_kg > (SELECT AVG(Waste_Generated_kg) FROM green_supply_chain)
AND ID > (SELECT ID FROM green_supply_chain ORDER BY ID LIMIT 1 OFFSET 99)
ORDER BY ID
LIMIT 10;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


6. Temporal Sustainability Trends
Compare quarterly average sustainability scores using generated dates (if no date column, simulate quarters using ID ranges).

In [ ]:
mycursor = mydb.cursor()

sql = """WITH SimulatedQuarters AS (
    SELECT *,
        CASE 
            WHEN ID % 4 = 0 THEN 'Q4' 
            WHEN ID % 3 = 0 THEN 'Q3' 
            WHEN ID % 2 = 0 THEN 'Q2' 
            ELSE 'Q1' 
        END AS Quarter
    FROM green_supply_chain
)
SELECT 
    Quarter,
    AVG(Sustainability_Score) AS Avg_Score,
    LAG(AVG(Sustainability_Score)) OVER (ORDER BY MIN(ID)) AS Prev_Quarter_Score
FROM SimulatedQuarters
GROUP BY Quarter;


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


7. Anti-Join for Anomaly Detection
Find products with below-average sustainability scores but above-average renewable energy usage.

In [ ]:
mycursor = mydb.cursor()

sql = """SELECT *
FROM green_supply_chain g
WHERE g.Sustainability_Score < (SELECT AVG(Sustainability_Score) FROM green_supply_chain)
AND NOT EXISTS (
    SELECT 1
    FROM green_supply_chain 
    WHERE Renewable_Energy_Percentage <= (SELECT AVG(Renewable_Energy_Percentage) FROM green_supply_chain)
    AND ID = g.ID
);


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


8. Dynamic Threshold Optimization
Create a function to calculate energy efficiency benchmarks that adjust based on product type.

In [ ]:
mycursor = mydb.cursor()

sql = """DELIMITER //
CREATE FUNCTION GetEnergyBenchmark(product_type VARCHAR(50)) 
RETURNS DECIMAL(10,2)
DETERMINISTIC
BEGIN
    DECLARE benchmark DECIMAL(10,2);
    SELECT AVG(Energy_Consumption_kWh) * 0.85 INTO benchmark
    FROM green_supply_chain
    WHERE Product_Type = product_type;
    RETURN benchmark;
END //
DELIMITER ;

SELECT ID, Energy_Consumption_kWh, GetEnergyBenchmark(Product_Type) AS Benchmark
FROM green_supply_chain;


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


9. Materialized View for Frequent Aggregates
Precompute and refresh daily stats for high-traffic sustainability dashboards.

In [ ]:
mycursor = mydb.cursor()

sql = """CREATE MATERIALIZED VIEW DailySustainability AS
SELECT 
    Product_Type,
    COUNT(*) AS Product_Count,
    AVG(CO2_Emissions_kg) AS Avg_CO2,
    SUM(Renewable_Energy_Percentage) AS Total_Renewables
FROM green_supply_chain
GROUP BY Product_Type;


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)


10. Multi-Criteria Indexing Challenge
Optimize this slow-running query without changing its logic:

In [ ]:
mycursor = mydb.cursor()

sql = """SELECT *
FROM green_supply_chain
WHERE Product_Type = 'Pharmaceutical'
AND (Renewable_Energy_Percentage > 40 OR Cost_USD < 5000)
ORDER BY Sustainability_Score DESC;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# -----

temp = list()
for i,data in enumerate(myresult):
    # print(x)
    temp.append(data)

import pandas as pd
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns = columns)
print(df)
